# **1. Perkenalan Dataset**


## Tentang Dataset
Dataset ini merupakan no-labelled data transaksi e-commerce berbasis di UK yang menjual hadiah dan peralatan rumah tangga untuk dewasa dan anak-anak melalui website sejak 2007. Karakteristik utama dataset:

- Periode: Data transaksi selama satu tahun
- Volume: 500,000 baris data
- Cakupan: Transaksi internasional dengan pelanggan dari berbagai negara
- Tipe Pelanggan: Konsumen langsung dan bisnis kecil yang membeli dalam jumlah besar

## Deskripsi Fitur-Fitur
Dataset memiliki 8 kolom dengan detail sebagai berikut:

- TransactionNo (kategorikal): Nomor unik 6-digit untuk setiap transaksi. Kode "C" mengindikasikan pembatalan
- Date (numerik): Tanggal generasi transaksi
- ProductNo (kategorikal): Karakter unik 5-6 digit untuk identifikasi produk
- Product (kategorikal): Nama produk/item
- Price (numerik): Harga per unit dalam pound sterling (£)
- Quantity (numerik): Kuantitas produk per transaksi. Nilai negatif menunjukkan pembatalan
- CustomerNo (kategorikal): Nomor unik 5-digit untuk identifikasi pelanggan
- Country (kategorikal): Negara tempat tinggal pelanggan

# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [22]:
# Data manipulation & analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Machine Learning & Clustering
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

# Distance metrics
from scipy.spatial.distance import cdist

# Feature Selection
from sklearn.feature_selection import SelectKBest, f_classif

# Configuration untuk tampilan visualisasi
# plt.style.use('seaborn')
sns.set_palette("husl")

# Mengatur opsi tampilan pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Mengabaikan warning
import warnings
warnings.filterwarnings('ignore')

# **3. Memuat Dataset**

In [14]:
df = pd.read_csv('https://media.githubusercontent.com/media/fxrdhan/Machine-Learning-Project/refs/heads/main/data/Sales%20Transaction%20v.4a.csv')

In [15]:
print("Dimensi Dataset:", df.shape)
print("\nJumlah baris:", df.shape[0])
print("Jumlah kolom:", df.shape[1])

Dimensi Dataset: (536350, 8)

Jumlah baris: 536350
Jumlah kolom: 8


In [16]:
print("\nTipe Data setiap Kolom:")
print(df.dtypes)


Tipe Data setiap Kolom:
TransactionNo     object
Date              object
ProductNo         object
ProductName       object
Price            float64
Quantity           int64
CustomerNo       float64
Country           object
dtype: object


In [17]:
print("\nJumlah Missing Values per Kolom:")
print(df.isnull().sum())


Jumlah Missing Values per Kolom:
TransactionNo     0
Date              0
ProductNo         0
ProductName       0
Price             0
Quantity          0
CustomerNo       55
Country           0
dtype: int64


In [18]:
print("\nInformasi Dataset:")
print(df.info())


Informasi Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536350 entries, 0 to 536349
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionNo  536350 non-null  object 
 1   Date           536350 non-null  object 
 2   ProductNo      536350 non-null  object 
 3   ProductName    536350 non-null  object 
 4   Price          536350 non-null  float64
 5   Quantity       536350 non-null  int64  
 6   CustomerNo     536295 non-null  float64
 7   Country        536350 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 32.7+ MB
None


In [20]:
print("\nSampel 5 Baris Pertama Dataset:")
df.head()


Sampel 5 Baris Pertama Dataset:


,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
0,581482,12/9/2019,22485,Set Of 2 Wooden Market Crates,21.470,12,17490.000,United Kingdom
1,581475,12/9/2019,22596,Christmas Star Wish List Chalkboard,10.650,36,13069.000,United Kingdom
2,581475,12/9/2019,23235,Storage Tin Vintage Leaf,11.530,12,13069.000,United Kingdom
3,581475,12/9/2019,23272,Tree T-Light Holder Willie Winkie,10.650,12,13069.000,United Kingdom
4,581475,12/9/2019,23239,Set Of 4 Knick Knack Tins Poppies,11.940,6,13069.000,United Kingdom


In [21]:
print("\nStatistik Deskriptif Kolom Numerik:")
df.describe()


Statistik Deskriptif Kolom Numerik:


,Price,Quantity,CustomerNo
count,536350.000,536350.000,536295.000
mean,12.662,9.919,15227.893
std,8.490,216.662,1716.583
min,5.130,-80995.000,12004.000
25%,10.990,1.000,13807.000
50%,11.940,3.000,15152.000
75%,14.090,10.000,16729.000
max,660.620,80995.000,18287.000


# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset. EDA bertujuan untuk:

1. **Memahami Struktur Data**
   - Tinjau jumlah baris dan kolom dalam dataset.  
   - Tinjau jenis data di setiap kolom (numerikal atau kategorikal).

2. **Menangani Data yang Hilang**  
   - Identifikasi dan analisis data yang hilang (*missing values*). Tentukan langkah-langkah yang diperlukan untuk menangani data yang hilang, seperti pengisian atau penghapusan data tersebut.

3. **Analisis Distribusi dan Korelasi**  
   - Analisis distribusi variabel numerik dengan statistik deskriptif dan visualisasi seperti histogram atau boxplot.  
   - Periksa hubungan antara variabel menggunakan matriks korelasi atau scatter plot.

4. **Visualisasi Data**  
   - Buat visualisasi dasar seperti grafik distribusi dan diagram batang untuk variabel kategorikal.  
   - Gunakan heatmap atau pairplot untuk menganalisis korelasi antar variabel.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [4]:
#Type your code here

# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning. Data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang perlu dilakukan, namun **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

In [5]:
#Type your code here

# **6. Pembangunan Model Clustering**

## **a. Pembangunan Model Clustering**

Pada tahap ini, Anda membangun model clustering dengan memilih algoritma yang sesuai untuk mengelompokkan data berdasarkan kesamaan. Berikut adalah **rekomendasi** tahapannya.
1. Pilih algoritma clustering yang sesuai.
2. Latih model dengan data menggunakan algoritma tersebut.

In [6]:
#Type your code here

## **b. Evaluasi Model Clustering**

Untuk menentukan jumlah cluster yang optimal dalam model clustering, Anda dapat menggunakan metode Elbow atau Silhouette Score.

Metode ini membantu kita menemukan jumlah cluster yang memberikan pemisahan terbaik antar kelompok data, sehingga model yang dibangun dapat lebih efektif. Berikut adalah **rekomendasi** tahapannya.
1. Gunakan Silhouette Score dan Elbow Method untuk menentukan jumlah cluster optimal.
2. Hitung Silhouette Score sebagai ukuran kualitas cluster.

In [7]:
#Type your code here

## **c. Feature Selection (Opsional)**

Silakan lakukan feature selection jika Anda membutuhkan optimasi model clustering. Jika Anda menerapkan proses ini, silakan lakukan pemodelan dan evaluasi kembali menggunakan kolom-kolom hasil feature selection. Terakhir, bandingkan hasil performa model sebelum dan sesudah menerapkan feature selection.

In [8]:
#Type your code here

## **d. Visualisasi Hasil Clustering**

Setelah model clustering dilatih dan jumlah cluster optimal ditentukan, langkah selanjutnya adalah menampilkan hasil clustering melalui visualisasi.

Berikut adalah **rekomendasi** tahapannya.
1. Tampilkan hasil clustering dalam bentuk visualisasi, seperti grafik scatter plot atau 2D PCA projection.

In [9]:
#Type your code here

## **e. Analisis dan Interpretasi Hasil Cluster**

Setelah melakukan clustering, langkah selanjutnya adalah menganalisis karakteristik dari masing-masing cluster berdasarkan fitur yang tersedia.

Berikut adalah **rekomendasi** tahapannya.
1. Analisis karakteristik tiap cluster berdasarkan fitur yang tersedia (misalnya, distribusi nilai dalam cluster).
2. Berikan interpretasi: Apakah hasil clustering sesuai dengan ekspektasi dan logika bisnis? Apakah ada pola tertentu yang bisa dimanfaatkan?

In [10]:
#Type your code here

Tulis hasil interpretasinya di sini.
1. Cluster 1:
2. Cluster 2:
3. Cluster 3:

# **7. Mengeksport Data**

Simpan hasilnya ke dalam file CSV.